In [148]:
import pandas as pd
import numpy as np
import altair as alt
import eco_style
import os
alt.themes.enable("light")  

ThemeRegistry.enable('light')

In [37]:
df = pd.read_csv("https://github.com/zonination/emperors/blob/master/emperors.csv?raw=true", encoding="latin").drop(columns=["index"])
df.to_csv("emperors.csv", index=False)

In [251]:
df = pd.read_csv("emperors.csv", encoding="latin")

df = df.rename(columns={
    "reign.end": "end",
    "reign.start": "start",
})

def date_to_float(date_str):
    if isinstance(date_str, str):
        try:
            parts = date_str.split('-')
            year = int(parts[0])
            month = int(parts[1])
            day = int(parts[2])
            # Approximate fractional year
            return year + (month - 1) / 12 + (day - 1) / 365.25
        except:
            return None
    return None

df["start"] = df["start"].apply(date_to_float)
df["end"] = df["end"].apply(date_to_float)

# apply a very little bit of delta to each end date to stop perfect  overlaps
df["end"] = df["end"] + np.random.uniform(0, 0.1, size=len(df))

# Make August' start negative
df.loc[df["name"] == "Augustus", "start"] = -26.041068

df = df.sort_values(by=["start", "end"], ascending=[True, True])

df = df.reset_index()
df['index'] = df.index + 1

# add in a 0,0 row
df = pd.concat([pd.DataFrame({"start": [-28], "end": [-28], "index": 0, "name": [""]}), df], ignore_index=True)


area = alt.Chart(df.query("end <= 306")).mark_area(
    color="#36b7b4",
    interpolate="step-before").encode(
        x=alt.X("end:Q", title="Year", 
                axis=alt.Axis(
                    labelExpr="datum.label + (datum.value < 0 ? ' BCE' : ' CE')",
                    values=[-25, 0, 50, 100, 150, 200, 250, 300],
                ),
                scale=alt.Scale(domain=[-28, 306])),
        y=alt.Y("index:Q", 
                axis=alt.Axis(
                    labelExpr="datum.label+(datum.value == 50 ? ' Emperors' : '')"
                ),
                title=""),
    )

# Shading for the crisis of the third century
crisis_shade = alt.Chart(pd.DataFrame({
    "start": [235],
    "end": [285],
    "label": ["Crisis of the Third Century"]
    })).mark_rect(
        fill="lightgray",
        opacity=0.5
    ).encode(
        x=alt.X("start:Q", title="Year"),
        x2=alt.X2("end:Q"),
    )

emperors_to_label = ["Augustus", "Diocletian"]
emperor_labels = alt.Chart(df.query("name in @emperors_to_label")).mark_text(
    align="left",
    baseline="middle",
    dy=alt.expr("datum.name === 'Augustus' ? -10 : 5"),
    dx=alt.expr("datum.name === 'Augustus' ? -15 : 20"),
    text=alt.expr("'('+datum.name+')'"),
    color="rgba(0, 0, 0, 0.4)",
).encode(
    x=alt.X("start:Q", title="Year"),
    y=alt.Y("index:Q", title=""),
)

other_labels_df = pd.DataFrame([
    {"label": "Crisis of the\nThird Century", "year": (284 + 235) / 2},
])

other_labels_df["label"] = other_labels_df["label"].apply(lambda x: x.split("\n"))

other_lables = alt.Chart(other_labels_df).mark_text(
    align="center",
    baseline="middle",
    dy=-9,
    dx=3,
    color="black",
).encode(
    x=alt.X("year:Q", title="Year"),
    y=alt.value(-10),
    text="label:N"
)


event_rules = alt.Chart(pd.DataFrame([
    {"label": "Year of the\nFour Emperors", "year": 69},
])).mark_rule(
    color="black",
    strokeDash=[3, 5],
).encode(
    x=alt.X("year:Q", title="Year"),
)

chart = crisis_shade+ area +emperor_labels+other_lables

chart = chart.properties(
    width=350,
    height=225
)

chart.save("cumulative_emperors.png", scale_factor=3)
chart.save("cumulative_emperors.json")
chart

alt.LayerChart(...)

In [256]:
df.query("index <= 49")

,start,end,index,name,name.full,birth,death,birth.cty,birth.prv,rise,cause,killer,dynasty,era,notes,verif.who
0,-28.000000,-28.000000,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-26.041068,14.663674,1,Augustus,IMPERATOR CAESAR DIVI FILIVS AVGVSTVS,0062-09-23,0014-08-19,Rome,Italia,Birthright,Assassination,Wife,Julio-Claudian,Principate,"birth, reign.start are BCE. Assign negative fo...",Reddit user zonination
2,14.713210,37.259539,2,Tiberius,TIBERIVS CAESAR DIVI AVGVSTI FILIVS AVGVSTVS,0041-11-16,0037-03-16,Rome,Italia,Birthright,Assassination,Other Emperor,Julio-Claudian,Principate,birth is BCE. Assign negative for correct ISO ...,Reddit user zonination
3,37.213210,41.084085,3,Caligula,GAIVS IVLIVS CAESAR AVGVSTVS GERMANICVS,0012-08-31,0041-01-24,Antitum,Italia,Birthright,Assassination,Senate,Julio-Claudian,Principate,assassination may have only involved the Praet...,Reddit user zonination
4,41.065708,54.874615,4,Claudius,TIBERIVS CLAVDIVS CAESAR AVGVSTVS GERMANICVS,0009-08-01,0054-10-13,Lugdunum,Gallia Lugdunensis,Birthright,Assassination,Wife,Julio-Claudian,Principate,birth is BCE. Assign negative for correct ISO ...,Reddit user zonination
5,54.782854,68.493618,5,Nero,NERO CLAVDIVS CAESAR AVGVSTVS GERMANICVS,0037-12-15,0068-06-09,Antitum,Italia,Birthright,Suicide,Senate,Julio-Claudian,Principate,NaN,Reddit user zonination
6,68.435832,69.124095,6,Galba,SERVIVS SVLPICIVS GALBA CAESAR AVGVSTVS,0002-12-24,0069-01-15,Terracina,Italia,Seized Power,Assassination,Other Emperor,Flavian,Principate,birth is BCE. Assign negative for correct ISO ...,Reddit user zonination
7,69.038330,69.384990,7,Otho,MARCVS SALVIVS OTHO CAESAR AVGVSTVS,0032-04-28,0069-04-16,Terentinum,Italia,Appointment by Praetorian Guard,Suicide,Other Emperor,Flavian,Principate,NaN,NaN
8,69.293806,70.053681,8,Vitellius,AVLVS VITELLIVS GERMANICVS AVGVSTVS,0015-09-24,0069-12-20,Rome,Italia,Seized Power,Assassination,Other Emperor,Flavian,Principate,NaN,NaN
9,69.971424,79.577188,9,Vespasian,TITVS FLAVIVS CAESAR VESPASIANVS AVGVSTVS,0009-11-17,0079-06-24,Falacrine,Italia,Seized Power,Natural Causes,Disease,Flavian,Principate,NaN,NaN


# Silver Content

alt.Chart(...)

In [257]:
dfs = []

for path in os.listdir("silver_content_velatrix_walker_1986"):
    df =  pd.read_csv(f"silver_content_velatrix_walker_1986/{path}", header=None, names=["year", "silver_content"])
    df['period'] = path.split(".")[0]
    dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df['year'] = df.year.round().astype(int)
df['silver_content'] = df.silver_content.round(1)/100

# Remove the 0.36-0.37 Severans - an error in the tracing
df = df.query("(silver_content > 0.38)")
points_df = df.copy()


points = alt.Chart(points_df).mark_circle().encode(
    x=alt.X("year:Q", title=""),
    y=alt.Y("silver_content:Q", title=""),
    color=alt.Color("period:N", title="", 
                    legend=alt.Legend(
                        orient="top",
                        #title="Walker (1986)",
                        title="",
                        titleFontWeight="normal",
                        titleOrient="top",
                        columns=4
                    ),
                    scale=alt.Scale(
                        domain=["Judio-Claudians", "Flavians", "Antonines", "Civil War (AD 69)", "Civil War (AD 193)", "Severans", "Third Century Crisis"],
                    )),
    tooltip=["year:Q", "silver_content:Q", "period:N"]
    
)

points

df = pd.read_csv("Silver_content-Pense - The Decline and Fall of The Roman Denarius.csv", header=None)
df.columns = ["year", "silver_content"]
df['year'] = df.year.round().astype(int)
df['silver_content'] = df.silver_content.round(0)/100
df['series'] = "Silver content (Pense, 1992)"

df['label'] = ''
df.iloc[-1,-1] = "Pense, 1992"

line = alt.Chart(df).mark_line(
    color="lightgrey",
    strokeWidth=2,
    ).encode(
        color=alt.Color("series:N", title="",
                        legend=None,
                        scale=alt.Scale(
                            domain=["Silver content (Pense, 1992)"],
                            range=["lightgrey"],
                        )),
        x=alt.X("year:Q", title="",
                axis=alt.Axis(
                    labelExpr="datum.label + (datum.value < 0 ? ' BCE' : ' CE')",
                ),
                scale=alt.Scale()),
        y=alt.Y("silver_content:Q", title="",
                axis=alt.Axis(
                    format="%",
                    labelExpr="datum.label + (datum.value == 1 ? ' Silver': '')"
                ),
                scale=alt.Scale(domain=[0, 1])),
    )

chart = (line + points).resolve_scale(
    color="independent"
).properties(
    width=350,
    height=225,
)

chart.save("silver_content.png", scale_factor=3)
chart.save("silver_content.json")
chart


alt.LayerChart(...)

In [258]:
df

,year,silver_content,series,label
0,-123,0.97,"Silver content (Pense, 1992)",
1,-115,0.97,"Silver content (Pense, 1992)",
2,-107,0.97,"Silver content (Pense, 1992)",
3,-102,0.97,"Silver content (Pense, 1992)",
4,-97,0.97,"Silver content (Pense, 1992)",
...,...,...,...,...
65,271,0.18,"Silver content (Pense, 1992)",
66,272,0.06,"Silver content (Pense, 1992)",
67,277,0.02,"Silver content (Pense, 1992)",
68,283,0.02,"Silver content (Pense, 1992)",


# Price edict

In [240]:
prices = [{
        "title": "Olive Oil\n(Extra virgin, ~540ml)",
        "price": 40,
        "group": "Oil"
    },
    {
        "title": "Olive Oil\n(Second pressing, ~540ml)",
        "price": 24,
        "group": "Oil"
    },
    {
        "title": "Olive Oil\n(Ordinary, ~540ml)",
        "price": 12,
        "group": "Oil"
    },
    {
        "title": "Pork\n(330g)",
        "price": 12,
        "group": "Meat"
    },
    {
        "title": "Beef\n(330g)",
        "price": 8,
        "group": "Meat"
    },
    {
        "title": "Sow's udder\n(330g)",
        "price": 20,
        "group": "Meat"
    },
    {
        "title": "Snails\n(20, largest)",
        "price": 20,
        "group": "Meat"
    },
    {
        "title": "Farm labourer\n(w/ maintenance, 1 day)",
        "price": 25,
        "group": "Labour"
    },
    {
        "title": "Carpenter\n(w/ maintenance, 1 day)",
        "price": 50,
        "group": "Labour"
    },
    {
        "title": "Figure painter\n(w/ maintenance, 1 day)",
        "price": 150,
        "group": "Labour"
    },
    {
        "title": "Wine\n(Year old, ~540ml)",
        "price": 16,
        "group": "Alcohol"
    },
    {
        "title": "Wheat beer\n(~540ml)",
        "price": 4,
        "group": "Alcohol"
    },
    {
        "title": "Spiced Wine\n(~540ml)",
        "price": 24,
        "group": "Alcohol"
    }
    
    
    ]

In [250]:
df = pd.DataFrame(prices)
df['title'] = df['title'].apply(lambda x: x.split("\n"))

# sort by group and then by price
df = df.sort_values(by=["group", "price"], ascending=[True, True])
df = df.reset_index()
df = df.rename(columns={"index": "rank"})


df_prices = df[df['group'] != "Labour"]
df_wages = df[df['group'] == "Labour"]

# Define common x encoding to ensure shared scale
x_shared = alt.X("price:Q", title="", axis=alt.Axis(
    labelAlign=alt.expr("datum.value == 160 ? 'left' : 'center'"),
    labelExpr="datum.label + ((datum.value == 160 | datum.value == -1) ? ' denarii' : '')"),)

# Price chart
chart_prices = alt.Chart(df_prices).mark_bar().encode(
    y=alt.Y("title:N", title="", sort=alt.EncodingSortField("rank:Q", order="ascending")),
    x=x_shared,
    color=alt.Color("group:N", legend=alt.Legend(
        orient="top",
        title="",
        values=["Oil", "Meat", "Alcohol", "Labour"],
    ))
).properties(
    title=alt.TitleParams(
        text="Prices",
        align="left",
        fontWeight="normal",
        anchor="start",
        fontSize=11,
        frame="group",
    ),
    height=250,
    width=350
)

# Wages chart
chart_wages = alt.Chart(df_wages).mark_bar().encode(
    y=alt.Y("title:N", title="", sort=alt.EncodingSortField("rank:Q", order="ascending")),
    x=x_shared,
    color=alt.Color("group:N")
).properties(
    title= alt.TitleParams(
        text="Wages",
        align="left",
        fontWeight="normal",
        anchor="start",
        fontSize=11,
        frame="group",
    ),
    height=100,
    width=350
)

chart = chart_prices & chart_wages
chart.save("prices_wages.png", scale_factor=3)
chart.save("prices_wages.json")
chart

alt.VConcatChart(...)

In [213]:
df

,rank,title,price,group
0,6,"[Farm labourer, (w/ maintenance, 1 day)]",25,Labour
1,7,"[Farm labourer, (w/ maintenance, 1 day)]",25,Labour
2,8,"[Carpenter, (w/ maintenance, 1 day)]",50,Labour
3,9,"[Figure painter, (w/ maintenance, 1 day)]",150,Labour
4,4,"[Beef, (330g)]",8,Meat
5,3,"[Pork, (330g)]",12,Meat
6,5,"[Sow's udder, (330g)]",20,Meat
7,2,"[Olive Oil, (Ordinary, ~540ml)]",12,Oil
8,1,"[Olive Oil, (Second pressing, ~540ml)]",24,Oil
9,0,"[Olive Oil, (Extra virgin, ~540ml)]",40,Oil


In [177]:
points_df.period.unique()

array(['Civil War (AD69)', 'Severans', 'Antonines', 'Judio-Claudians',
       'Flavians', 'Civil War (AD 193)', 'Third Century Crisis'],
      dtype=object)

In [97]:
area

alt.Chart(...)

In [88]:
df['length'] = df['end'] - df['start']
df.sort_values(by='length', ascending=False).query("end<306")

,start,end,index,name,name.full,birth,death,birth.cty,birth.prv,rise,cause,killer,dynasty,era,notes,verif.who,length
1,-26.041068,14.673420,1,Augustus,IMPERATOR CAESAR DIVI FILIVS AVGVSTVS,0062-09-23,0014-08-19,Rome,Italia,Birthright,Assassination,Wife,Julio-Claudian,Principate,"birth, reign.start are BCE. Assign negative fo...",Reddit user zonination,40.714488
15,138.524641,161.250270,15,Antonius Pius,CAESAR TITVS AELIVS HADRIANVS ANTONINVS AVGVST...,0086-09-19,0161-03-07,Lanuvium,Italia,Birthright,Natural Causes,Disease,Nerva-Antonine,Principate,NaN,NaN,22.725629
2,14.713210,37.275983,2,Tiberius,TIBERIVS CAESAR DIVI AVGVSTI FILIVS AVGVSTVS,0041-11-16,0037-03-16,Rome,Italia,Birthright,Assassination,Other Emperor,Julio-Claudian,Principate,birth is BCE. Assign negative for correct ISO ...,Reddit user zonination,22.562773
14,117.610712,138.605464,14,Hadrian,CAESAR PVBLIVS AELIVS TRAIANVS HADRIANVS AVGVSTVS,0076-01-24,0138-07-10,Italica,Hispania Baetica,Birthright,Natural Causes,Heart Failure,Nerva-Antonine,Principate,NaN,NaN,20.994752
49,284.885352,305.394717,49,Diocletian,CAESAR GAIVS AVRELIVS VALERIVS DIOCLETIANVS AV...,0244-12-22,0311-12-03,Salona,Dalmatian,Seized Power,Natural Causes,Disease,Constantinian,Dominate,NaN,NaN,20.509365
13,98.073922,117.604588,13,Trajan,CAESAR MARCVS VLPIVS NERVA TRAIANVS AVGVSTVS,0053-09-18,0117-08-07,Italica,Hispania Baetica,Birthright,Natural Causes,Disease,Nerva-Antonine,Principate,NaN,NaN,19.530666
22,198.000000,217.269310,22,Caracalla,CAESAR MARCVS AVRELIVS SEVERVS ANTONINVS PIVS ...,0188-04-04,0217-04-08,Lugdunum,Gallia Lugdunensis,Birthright,Assassination,Other Emperor,Severan,Principate,reign.start is only valid for year,NaN,19.269310
50,286.250000,305.335887,50,Maximian,CAESAR MARCVS AVRELIVS VALERIVS MAXIMIANVS AVG...,0250-01-01,0310-07-15,Sirmium,Pannonia,Appointment by Emperor,Suicide,Other Emperor,Constantinian,Dominate,"birth, death are estimates",NaN,19.085887
17,161.183094,180.219517,17,Marcus Aurelius,CAESAR MARCVS AVRELIVS ANTONINVS AVGVSTVS,0121-04-26,0180-03-17,Rome,Italia,Birthright,Natural Causes,Disease,Nerva-Antonine,Principate,NaN,NaN,19.036423
21,193.271903,211.160744,21,Septimus Severus,CAESAR LVCIVS SEPTIMIVS SEVERVS PERTINAX AVGVSTVS,0145-04-11,0211-02-04,Leptis Magna,Libya,Seized Power,Natural Causes,Disease,Severan,Principate,NaN,NaN,17.888841


In [47]:
df.query("name == 'Diocletian'").head(1)

,index,name,name.full,birth,death,birth.cty,birth.prv,rise,start,end,cause,killer,dynasty,era,notes,verif.who
48,48,Diocletian,CAESAR GAIVS AVRELIVS VALERIVS DIOCLETIANVS AV...,0244-12-22,0311-12-03,Salona,Dalmatian,Seized Power,284.885352,305.34273,Natural Causes,Disease,Constantinian,Dominate,NaN,NaN


In [30]:
pd.to_datetime("26-01-16", errors='coerce')

Timestamp('2016-01-26 00:00:00')

In [33]:
df

,index,name,name.full,birth,death,birth.cty,birth.prv,rise,start,end,cause,killer,dynasty,era,notes,verif.who
0,1,Augustus,IMPERATOR CAESAR DIVI FILIVS AVGVSTVS,0062-09-23,0014-08-19,Rome,Italia,Birthright,26.041068,14.632615,Assassination,Wife,Julio-Claudian,Principate,"birth, reign.start are BCE. Assign negative fo...",Reddit user zonination
1,2,Tiberius,TIBERIVS CAESAR DIVI AVGVSTI FILIVS AVGVSTVS,0041-11-16,0037-03-16,Rome,Italia,Birthright,14.713210,37.207734,Assassination,Other Emperor,Julio-Claudian,Principate,birth is BCE. Assign negative for correct ISO ...,Reddit user zonination
2,3,Caligula,GAIVS IVLIVS CAESAR AVGVSTVS GERMANICVS,0012-08-31,0041-01-24,Antitum,Italia,Birthright,37.213210,41.062971,Assassination,Senate,Julio-Claudian,Principate,assassination may have only involved the Praet...,Reddit user zonination
3,4,Claudius,TIBERIVS CLAVDIVS CAESAR AVGVSTVS GERMANICVS,0009-08-01,0054-10-13,Lugdunum,Gallia Lugdunensis,Birthright,41.065708,54.782854,Assassination,Wife,Julio-Claudian,Principate,birth is BCE. Assign negative for correct ISO ...,Reddit user zonination
4,5,Nero,NERO CLAVDIVS CAESAR AVGVSTVS GERMANICVS,0037-12-15,0068-06-09,Antitum,Italia,Birthright,54.782854,68.438569,Suicide,Senate,Julio-Claudian,Principate,NaN,Reddit user zonination
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Valentinian I,FLAVIVS VALENTINIANVS AVGVSTVS,0321-07-03,0375-11-17,Cibalae,Pannonia,Election,364.151780,375.877139,Natural Causes,Aneurism,Valentinian,Dominate,NaN,Reddit user ironicosity
64,65,Valens,FLAVIVS IVLIVS VALENS AVGVSTVS,0328-01-01,0378-08-09,Cibalae,Pannonia,Birthright,364.240589,378.605236,Died in Battle,Opposing Army,Valentinian,Dominate,birth is only estimate.,Reddit user ironicosity
65,66,Gratian,FLAVIVS GRATIANVS AVGVSTVS,0359-04-18,0383-08-25,Sirmium,Pannonia,Birthright,367.591547,383.649042,Assassination,Own Army,Valentinian,Dominate,birth may also be 0359-05-23,Reddit user ironicosity
66,67,Valentinian II,FLAVIVS VALENTINIANVS INVICTVS AVGVSTVS,0371-01-01,0392-05-15,Milan,Italia,Birthright,375.877139,392.371663,Suicide,Unknown,Valentinian,Dominate,birth is only estimate.,Reddit user ironicosity
